# Just exploring some potential datasets

In [59]:
from ucimlrepo import fetch_ucirepo, list_available_datasets
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import numpy as np

#list_avaliable_datasets()


In [60]:
# Dataset that seems useful
dataset_id = {"iris": 53, 
              "heart_disease": 45,
              "wine_quality": 186,
              "breast_cancer_wisconsin": 17,
              "car_evaluation": 19, # Not good maybe remove
              "abalone": 1,
              "mushroom": 73, # a lot of categorical data
              "statlog" : 144, # german credit
              "student_performance" : 320
              }

TEST_SIZE = 0.2
RANDOM_SEED = 42
K_FOLDS = 3

# Load data

### Turn categorical features into numerical

In [61]:
def encode_categorical_features(X, encoder):
    X = encoder.fit_transform(X)
    return X

In [62]:
def import_dataset(uci_id, encoder):
    dataset = fetch_ucirepo(id=uci_id) 
    print(dataset.metadata)
    data_types = dataset.variables["type"]
    data_names = dataset.variables["name"]
    data_roles = dataset.variables["role"]
    X = dataset.data.features
    y = dataset.data.targets
    print(data_names)
    print("Column names in X:", X.columns.tolist())
    categorical_columns = []
    #categorical_columns = [data_names[i] for i, type in enumerate(data_types) if (type == "Categorical" or type == "Binary") and data_names[i] != "class"]
    for i, type in enumerate(data_types):
        # print(data_names[i], type, data_roles[i])
        if (type == "Categorical" or type == "Binary") and data_names[i] != "class" and data_roles[i] == "Feature":
            categorical_columns.append(data_names[i])
    X.loc[:, categorical_columns] = encode_categorical_features(X[categorical_columns], encoder)
    return X, y


In [63]:
ordinal_encoder = OrdinalEncoder()
X, y = import_dataset(dataset_id["breast_cancer_wisconsin"], ordinal_encoder)
print(type(X))
print(y)
print(X)

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'ID': 230, 'type': 'NATIVE', 'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'venue': 'Electronic imaging', 'year': 1993, 'journal': None, 'DOI': '1

/var/folders/s9/bg_8ddhj38x2g3svq29mlmcc0000gn/T/ipykernel_78577/2690621505.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, categorical_columns] = encode_categorical_features(X[categorical_columns], encoder)


# Train test split

In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## Cross validation
- #TODO Look up TimeSeriesSplit for time series data
- #TODO just standard scaling to test if it works

In [67]:
log_reg = LogisticRegression(max_iter=1000)

# Ravel to convert from (len, 1) shape to (len,), warning from sk-learn
Y_train = np.ravel(Y_train)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

scores = cross_val_score(
    log_reg, X_scaled, Y_train, scoring='accuracy', cv=5)
# accuracy
print('Accuracy: %.3f ,\nStandard Deviations :%.3f' %
      (np.mean(scores), np.std(scores)))

#TODO check if this is better implementation
# k_folds  = KFold(n_splits=K_FOLDS)
# for train_idx, valid_idx in k_folds.split(X_train):
#     continue

Accuracy: 0.974 ,
Standard Deviations :0.018


In [66]:
# Testing importing the first 25 UCI datasets
# for i in range(1, 25):
#     try:
#         X, y = import_dataset(i, ordinal_encoder)
#     except Exception as e:
#         if "DatasetNotFoundError" in str(type(e)):
#             print(f"Dataset {i} is not available for import. Skipping. Error: {str(e)}")
#         else:
#             print(f"An unexpected error occurred with dataset {i}: {str(e)}")